In [35]:
##Branch and Bound!

import pandas as pd

import statsmodels.formula.api as smf
import numpy as np
import scipy.stats
from scipy.stats import norm


data = pd.read_csv("C:/Users/Praktikant1/Desktop/Projekte/Branch and Bound/Branch-and-Bound/dataset.csv",sep=";").astype(float)



In [36]:
#variables and speficicatons
headers = list(data)

Y = headers[0]
X = headers[1:]

In [37]:
from sklearn.linear_model import LinearRegression  
regressor = LinearRegression()  
R = regressor.fit(data[X],data[Y])  

coeff_df = pd.DataFrame(regressor.coef_, data[X].columns, columns=['Coefficient'])  
coeff_df  
#print(R)


,Coefficient
SEX,0.809046
INCOME,0.071686
EDUCATION,-0.044520
AGE,0.009725
LOCATION,0.084003


In [38]:
## exhaustive search?

class bbalgorithm: 
    def __init__(self, dependant, indepedant):
        self.depedant = dependant
        self.indepedant = independat
    
    def branch():
        
        pass
    
    def bound():
        
        pass
    
